In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:

# import json
# def convert_to_bilou_format(raw_data):
#     bilou_data = {}

#     for index, case_data in enumerate(raw_data):
#         raw_words = case_data["raw_words"]
#         words = case_data["words"]
#         aspects = case_data["aspects"]
#         opinions = case_data["opinions"]

#         bilou_labels = ['O'] * len(words)

#         for aspect in aspects:
#             for idx,i in enumerate(range(aspect["from"], aspect["to"])):
#                 if i == aspect["from"]:
#                     bilou_labels[i] = 'B'
#                 else:
#                     bilou_labels[i] = 'I'

#         # for opinion in opinions:
#         #     for idx,i in enumerate(range(opinion["from"], opinion["to"])):
#         #         bilou_labels[i] = 'B_' + opinion["term"][idx]

#         bilou_data[index] = {
#             "text": raw_words,
#             "labels": bilou_labels
#         }
#     # print(bilou_data)
#     return bilou_data

# # Your JSON data
# # Read input data from a JSON file
# for i in ["Train","Test","Val"]:
#   with open(f"D:/Downloads/Laptop_Review_{i}.json", 'r') as json_file:
    
#       raw_data = json.load(json_file)

#   # Convert to BILOU format
#   bilou_data = convert_to_bilou_format(raw_data)

#   # Save the output to a JSON file
#   with open(f'bilou_data_{i}.json', 'w') as json_file:
#       json.dump(bilou_data, json_file, indent=2)


In [3]:
# !pip install fasttext

In [4]:
import torch
from torch.utils.data import Dataset,DataLoader
import gensim.downloader as api
from torchtext.vocab import GloVe
import fasttext
import numpy as np
from torchtext.vocab import GloVe,FastText
import fasttext.util
import json
class SentimentAnalysisDataset(Dataset):
    def __init__(self, json_path, embedding_type='word2vec',load=True):
        with open(json_path, 'r') as file:
            self.data = json.load(file)

        self.embedding_type = embedding_type
        if load:
          self.embedding_model =self.load_embedding_model()
        else:
          self.embedding_model = None

    def load_embedding_model(self):
        if self.embedding_type == 'word2vec':
            # Download the pre-trained Word2Vec model
            return api.load('word2vec-google-news-300')
        elif self.embedding_type == 'glove':
            # Download the pre-trained GloVe model (6B tokens, 300d)
            return GloVe(name='6B', dim=300)
        elif self.embedding_type == 'fasttext':
            # Load the pre-trained FastText model
            
            return FastText(language='en')
        else:
            raise ValueError(f"Unsupported embedding type: {self.embedding_type}")

    def text_to_embeddings(self, text):
        maxlen = 100
        if self.embedding_type == 'word2vec':
            # Word2Vec embeddings
            embeddings = [self.embedding_model[word] if word in self.embedding_model else torch.zeros(self.embedding_model.vector_size) for word in text.split() ]
            # print(np.stack(embeddings).shape)
            embeddings = [torch.full((self.embedding_model.vector_size,),-1000.0)] + embeddings + [torch.full((self.embedding_model.vector_size,),+1000.0)]

            # print('##',np.stack([torch.full((self.embedding_model.vector_size,),-1000.0)] + embeddings + [torch.full((self.embedding_model.vector_size,),+1000.0)]  ).shape)

            for i in range(100-len(embeddings)):
              embeddings.append(torch.full((self.embedding_model.vector_size,),-1.0))


        elif self.embedding_type == 'glove':
            # GloVe embeddings
            embeddings = [self.embedding_model[word] if word in self.embedding_model else torch.zeros(sentiment_dataset.embedding_model['a'].shape[0]) for word in text.split() ]
            # print(np.stack(embeddings).shape)
            embeddings = [torch.full((sentiment_dataset.embedding_model['a'].shape[0],),-1000.0)] + embeddings + [torch.full((sentiment_dataset.embedding_model['a'].shape[0],),+1000.0)]

            # print('##',np.stack([torch.full((self.embedding_model.vector_size,),-1000.0)] + embeddings + [torch.full((self.embedding_model.vector_size,),+1000.0)]  ).shape)

            for i in range(100-len(embeddings)):
              embeddings.append(torch.full((sentiment_dataset.embedding_model['a'].shape[0],),-1.0))

        elif self.embedding_type == 'fasttext':
            # FastText embeddings
            embeddings = [self.embedding_model[word] for word in text.split() ]
            # print(np.stack(embeddings).shape)
            embeddings = [torch.full((sentiment_dataset.embedding_model['a'].shape[0],),-1000.0)] + embeddings + [torch.full((sentiment_dataset.embedding_model['a'].shape[0],),+1000.0)]

            # print('##',np.stack([torch.full((self.embedding_model.vector_size,),-1000.0)] + embeddings + [torch.full((self.embedding_model.vector_size,),+1000.0)]  ).shape)

            for i in range(100-len(embeddings)):
              embeddings.append(torch.full((sentiment_dataset.embedding_model['a'].shape[0],),-1.0))
        else:
            raise ValueError(f"Unsupported embedding type: {self.embedding_type}")
        # print()
        return np.stack(embeddings)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):


        index = str(index)
        text = self.data[index]["text"]
        labels = self.data[index]["labels"]

        # Convert text to embeddings
        text_embeddings = torch.tensor(self.text_to_embeddings(text))
        # print(text_embeddings.shape)
        # torch.stack([torch.full((1,text_embeddings.shape[1]),-1000),text_embeddings, [torch.full((1,text_embeddings.shape[1]),1000)])
        current_length = len(labels)

        labels = ['<START>'] + labels + ['<STOP>']

        sent_lengths =torch.tensor(len(labels))

        max_length = 100
        labels = labels + ['<PAD>'] * (max_length - (current_length+2))

        # Convert labels to numerical format if needed
        label_mapping = {'O': 0, 'B': 1, 'I':2,'<START>':3,'<STOP>':4,'<PAD>':5}                               # bind it to self__________________________
        numerical_labels = [label_mapping[label] for label in labels ]


        # Pad the sequence to the maximum length

        # Convert labels to PyTorch tensor
        labels_tensor = torch.tensor(numerical_labels)
        mask = torch.hstack([torch.full((text_embeddings.shape[0],),True),torch.full((100-text_embeddings.shape[0],),False)])
        # print(labels_tensor.shape,text_embeddings.shape,mask.shape)
        return text_embeddings, labels_tensor, mask,sent_lengths

# Example usage
json_path = 'ATE_train.json'
embedding_type = 'fasttext'
sentiment_dataset = SentimentAnalysisDataset(json_path, embedding_type)
# sentiment_dataset.embedding_model= temp.embedding_model
# Accessing a sample from the dataset
sample_text_embeddings, sample_labels, mask,s_len = sentiment_dataset[0]
print("Sample Text Embeddings:", sample_text_embeddings)
print("Sample Labels:", sample_labels)


.vector_cache\wiki.en.vec: 6.60GB [02:51, 38.4MB/s]                                                                    
100%|█████████████████████████████████████████████████████████████████████| 2519370/2519370 [04:04<00:00, 10313.68it/s]


Sample Text Embeddings: tensor([[-1.0000e+03, -1.0000e+03, -1.0000e+03,  ..., -1.0000e+03,
         -1.0000e+03, -1.0000e+03],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [-1.3457e-01, -1.5672e-01,  6.4009e-03,  ...,  1.1805e-01,
          2.6181e-01,  1.0162e-01],
        ...,
        [-1.0000e+00, -1.0000e+00, -1.0000e+00,  ..., -1.0000e+00,
         -1.0000e+00, -1.0000e+00],
        [-1.0000e+00, -1.0000e+00, -1.0000e+00,  ..., -1.0000e+00,
         -1.0000e+00, -1.0000e+00],
        [-1.0000e+00, -1.0000e+00, -1.0000e+00,  ..., -1.0000e+00,
         -1.0000e+00, -1.0000e+00]])
Sample Labels: tensor([3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 4, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5

In [5]:
json_path = 'ATE_val.json'
sentiment_dataset_val = SentimentAnalysisDataset(json_path, embedding_type,load=False)
sentiment_dataset_val.embedding_model = sentiment_dataset.embedding_model

In [6]:
json_path = 'ATE_test.json'
sentiment_dataset_test = SentimentAnalysisDataset(json_path, embedding_type,load=False)
sentiment_dataset_test.embedding_model = sentiment_dataset.embedding_model

In [18]:
sample_text_embeddings.shape

torch.Size([100, 300])

In [19]:
sample_labels.shape

torch.Size([100])

In [20]:
mask.shape

torch.Size([256, 100])

In [21]:
batch_size  = 256
dataloader = DataLoader(sentiment_dataset, batch_size=batch_size, shuffle=True)

In [22]:
batch_size  = 256
dataloader_val = DataLoader(sentiment_dataset_val, batch_size=batch_size, shuffle=True)

In [23]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence


class BiLSTMCRF(nn.Module):
    def __init__(self, tag_vocab, dropout_rate=0.5, embed_size=300, hidden_size=256):
        """ Initialize the model
        Args:
            sent_vocab (Vocab): vocabulary of words
            tag_vocab (Vocab): vocabulary of tags
            embed_size (int): embedding size
            hidden_size (int): hidden state size
        """
        super(BiLSTMCRF, self).__init__()

        self.dropout_rate = dropout_rate
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        # self.sent_vocab = sent_vocab
        self.tag_vocab = tag_vocab
        # self.embedding = nn.Embedding(len(sent_vocab), embed_size) print
        self.dropout = nn.Dropout(dropout_rate)
        self.encoder = nn.LSTM(input_size=embed_size, hidden_size=hidden_size, bidirectional=True)
        self.hidden2emit_score = nn.Linear(hidden_size * 2, len(self.tag_vocab))
        self.transition = nn.Parameter(torch.randn(len(self.tag_vocab), len(self.tag_vocab)))  # shape: (K, K)

    def forward(self, sentences,mask, tags, sen_lengths):
        """
        Args:
            sentences (tensor): sentences, shape (b, len). Lengths are in decreasing order, len is the length
                                of the longest sentence
            tags (tensor): corresponding tags, shape (b, len)
            sen_lengths (list): sentence lengths
        Returns:
            loss (tensor): loss on the batch, shape (b,)
        """
        # mask = (sentences != self.sent_vocab[self.sent_vocab.PAD])  # shape: (b, len)                        #$$$$$$$$$$$$$$$$$$$__________________
        sentences = sentences.transpose(0, 1)  # shape: (len, b)
        # print("forword--1",sentences.shape)
        # sentences = self.embedding(sentences)  # shape: (len, b, e)
        emit_score = self.encode(sentences, sen_lengths)  # shape: (b, len, K)
        # print("forword--2",sentences.shape)
        loss = self.cal_loss(tags, mask, emit_score)  # shape: (b,)
        return loss

    def encode(self, sentences, sent_lengths):
        """ BiLSTM Encoder
        Args:
            sentences (tensor): sentences with word embeddings, shape (len, b, e)
            sent_lengths (list): sentence lengths
        Returns:
            emit_score (tensor): emit score, shape (b, len, K)
        """
        # padded_sentences = pack_padded_sequence(sentences, sent_lengths)
        hidden_states, _ = self.encoder(sentences)
        # print(hidden_states.shape,"(((())))")
        hidden_states = hidden_states.permute(1,0,2)
        # hidden_states, _ = pad_packed_sequence(hidden_states, batch_first=True)  # shape: (b, len, 2h)
        # print(hidden_states.shape)
        emit_score = self.hidden2emit_score(hidden_states)  # shape: (b, len, K)
        emit_score = self.dropout(emit_score)  # shape: (b, len, K)
        return emit_score

    # def encode(self, sentences, sent_lengths):
    #   """ BiLSTM Encoder
    #   Args:
    #       sentences (tensor): sentences with word embeddings, shape (len, b, e)
    #       sent_lengths (list): sentence lengths
    #   Returns:
    #       emit_score (tensor): emit score, shape (b, len, K)
    #   """
    #   sorted_lengths, sorted_idx = torch.sort(sent_lengths, descending=True)
    #   sorted_sentences = sentences[:, sorted_idx, :]  # Sort the sentences based on lengths
    #   packed_sentences = pack_padded_sequence(sorted_sentences, sorted_lengths)
    #   hidden_states, _ = self.encoder(packed_sentences)
    #   hidden_states, _ = pad_packed_sequence(hidden_states, batch_first=True)  # shape: (b, len, 2h)
    #   emit_score = self.hidden2emit_score(hidden_states)  # shape: (b, len, K)
    #   emit_score = self.dropout(emit_score)  # shape: (b, len, K)
    #   return emit_score

    def cal_loss(self, tags, mask, emit_score):
        """ Calculate CRF loss
        Args:
            tags (tensor): a batch of tags, shape (b, len)
            mask (tensor): mask for the tags, shape (b, len), values in PAD position is 0
            emit_score (tensor): emit matrix, shape (b, len, K)
        Returns:
            loss (tensor): loss of the batch, shape (b,)
        """
        batch_size, sent_len = tags.shape
        # calculate score for the tags
        score = torch.gather(emit_score, dim=2, index=tags.unsqueeze(dim=2)).squeeze(dim=2)  # shape: (b, len)
        score[:, 1:] += self.transition[tags[:, :-1], tags[:, 1:]]
        total_score = (score * mask.type(torch.float)).sum(dim=1)  # shape: (b,)
        # calculate the scaling factor
        d = torch.unsqueeze(emit_score[:, 0], dim=1)  # shape: (b, 1, K)
        fix_length = 100
        # for i in range(1, sent_len):
        for i in range(1, fix_length):
            n_unfinished = mask[:, i].sum()
            d_uf = d[: n_unfinished]  # shape: (uf, 1, K)
            emit_and_transition = emit_score[: n_unfinished, i].unsqueeze(dim=1) + self.transition  # shape: (uf, K, K)
            log_sum = d_uf.transpose(1, 2) + emit_and_transition  # shape: (uf, K, K)
            max_v = log_sum.max(dim=1)[0].unsqueeze(dim=1)  # shape: (uf, 1, K)
            log_sum = log_sum - max_v  # shape: (uf, K, K)
            d_uf = max_v + torch.logsumexp(log_sum, dim=1).unsqueeze(dim=1)  # shape: (uf, 1, K)
            d = torch.cat((d_uf, d[n_unfinished:]), dim=0)
        d = d.squeeze(dim=1)  # shape: (b, K)
        max_d = d.max(dim=-1)[0]  # shape: (b,)
        d = max_d + torch.logsumexp(d - max_d.unsqueeze(dim=1), dim=1)  # shape: (b,)
        llk = total_score - d  # shape: (b,)
        loss = -llk  # shape: (b,)
        return loss


    def predict(self, sentences, mask, sen_lengths):
        """
        Args:
            sentences (tensor): sentences, shape (b, len). Lengths are in decreasing order, len is the length
                                of the longest sentence
            sen_lengths (list): sentence lengths
        Returns:
            tags (list[list[str]]): predicted tags for the batch
        """
        batch_size = sentences.shape[0]

        w = mask
        sentences = sentences.transpose(0, 1)

        emit_score = self.encode(sentences, sen_lengths)

        # Initialize the tags with all possible tag indices for each sentence in the batch
        tags = [[[i] for i in range(len(self.tag_vocab))]] * batch_size  # list, shape: (b, K, 1)

        # Initialize the first column of the dynamic programming matrix
        d = torch.unsqueeze(emit_score[:, 0], dim=1)  # shape: (b, 1, K)

        # Use a fixed length (e.g., 100) instead of max(sen_lengths)
        fixed_length = 100

        # Iterate over the remaining columns of the dynamic programming matrix
        for i in range(1, fixed_length):
            # Calculate the number of unfinished sentences at the current position
            n_unfinished = mask[:, i].sum()

            # Slice the dynamic programming matrix for the unfinished sentences
            d_uf = d[: n_unfinished]  # shape: (uf, 1, K)

            # Compute emission and transition scores for the current position
            emit_and_transition = self.transition + emit_score[: n_unfinished, i].unsqueeze(dim=1)  # shape: (uf, K, K)

            # Compute the new values for the dynamic programming matrix
            new_d_uf = d_uf.transpose(1, 2) + emit_and_transition  # shape: (uf, K, K)

            # Update the dynamic programming matrix and get the indices of maximum values
            d_uf, max_idx = torch.max(new_d_uf, dim=1)
            max_idx = max_idx.tolist()  # list, shape: (nf, K)

            # Update the tags for the unfinished sentences
            tags[: n_unfinished] = [[tags[b][k] + [j] for j, k in enumerate(max_idx[b])] for b in range(n_unfinished)]

            # Concatenate the new values to the dynamic programming matrix
            d = torch.cat((torch.unsqueeze(d_uf, dim=1), d[n_unfinished:]), dim=0)  # shape: (b, 1, K)

        # Remove the singleton dimension to get the final dynamic programming matrix
        d = d.squeeze(dim=1)  # shape: (b, K)

        # Get the indices of the maximum values in the final column of the matrix
        _, max_idx = torch.max(d, dim=1)  # shape: (b,)
        max_idx = max_idx.tolist()

        # Extract the predicted tags based on the maximum indices
        tags = [tags[b][k] for b, k in enumerate(max_idx)]

        # Print the predicted tags and sentence lengths for debugging
        # print(tags, sen_lengths, '((()))')

        return tags

    # def predict(self, sentences,mask, sen_lengths):
    #     """
    #     Args:
    #         sentences (tensor): sentences, shape (b, len). Lengths are in decreasing order, len is the length
    #                             of the longest sentence
    #         sen_lengths (list): sentence lengths
    #     Returns:
    #         tags (list[list[str]]): predicted tags for the batch
    #     """
    #     batch_size = sentences.shape[0]
    #     # w = (sentences != self.sent_vocab[self.sent_vocab.PAD])  # shape: (b, len)
    #     w = mask
    #     sentences = sentences.transpose(0, 1)  # shape: (len, b)
    #     # sentences = self.embedding(sentences)  # shape: (len, b, e)
    #     emit_score = self.encode(sentences, sen_lengths)  # shape: (b, len, K)
    #     tags = [[[i] for i in range(len(self.tag_vocab))]] * batch_size  # list, shape: (b, K, 1)
    #     d = torch.unsqueeze(emit_score[:, 0], dim=1)  # shape: (b, 1, K)
    #     for i in range(1, sen_lengths[0]):
    #         n_unfinished = mask[:, i].sum()
    #         d_uf = d[: n_unfinished]  # shape: (uf, 1, K)
    #         emit_and_transition = self.transition + emit_score[: n_unfinished, i].unsqueeze(dim=1)  # shape: (uf, K, K)
    #         new_d_uf = d_uf.transpose(1, 2) + emit_and_transition  # shape: (uf, K, K)
    #         d_uf, max_idx = torch.max(new_d_uf, dim=1)
    #         max_idx = max_idx.tolist()  # list, shape: (nf, K)
    #         tags[: n_unfinished] = [[tags[b][k] + [j] for j, k in enumerate(max_idx[b])] for b in range(n_unfinished)]
    #         d = torch.cat((torch.unsqueeze(d_uf, dim=1), d[n_unfinished:]), dim=0)  # shape: (b, 1, K)
    #     d = d.squeeze(dim=1)  # shape: (b, K)
    #     _, max_idx = torch.max(d, dim=1)  # shape: (b,)
    #     max_idx = max_idx.tolist()
    #     tags = [tags[b][k] for b, k in enumerate(max_idx)]
    #     print(tags,sen_lengths,'((()))')
    #     return tags

    # def save(self, filepath):
    #     params = {
    #         'sent_vocab': self.sent_vocab,
    #         'tag_vocab': self.tag_vocab,
    #         'args': dict(dropout_rate=self.dropout_rate, embed_size=self.embed_size, hidden_size=self.hidden_size),
    #         'state_dict': self.state_dict()
    #     }
    #     torch.save(params, filepath)


# def main():
#     sent_vocab = Vocab.load('./vocab/sent_vocab.json')
#     tag_vocab = Vocab.load('./vocab/tag_vocab.json')
#     train_data, dev_data = utils.generate_train_dev_dataset('./data/train.txt', sent_vocab, tag_vocab)
#     device = torch.device('cpu')
#     model = BiLSTMCRF(sent_vocab, tag_vocab)
#     model.to(device)
#     model.save('./model/model.pth')
#     model = model.load('./model/model.pth', device)


# if __name__ == '__main__':
    # main()


In [24]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
tag_to_ix ={'O': 0, 'B': 1, 'I':2,'<START>':3,'<STOP>':4,'<PAD>':5}
# tag_vocab, dropout_rate=0.5, embed_size=300, hidden_size=256)
model  = BiLSTMCRF(tag_to_ix,dropout_rate=0.5, embed_size=300, hidden_size=256)

import torch.optim as optim
learning_rate = 0.0001
momentum = 0.9  # Optional: You can adjust the momentum term

# Create an instance of the SGD optimizer
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)


In [25]:
# from tqdm import tqdm
# import torch

# # Assuming you have a BiLSTM_CRF model, a dataloader, and an optimizer
# # Also assuming you have defined the necessary variables (e.g., vocab_size, tag_to_ix, etc.)

# # Move the model to GPU
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

# # Check predictions before training
# for epoch in range(300):
#     total_loss = 0
#     correct_predictions = 0
#     total_sentences = 0

#     # Wrap your dataloader with tqdm for a progress bar
#     for sentence_in, targets, mask, sen_lengths in tqdm(dataloader, desc=f'Epoch {epoch + 1}/{300}', leave=False):
#         # Move input data to GPU
#         sentence_in, targets, mask, sen_lengths = sentence_in.to(device), targets.to(device), mask.to(device), sen_lengths

#         model.zero_grad()

#         # Forward pass
#         loss = model(sentence_in, mask, targets, sen_lengths)

#         # Backward pass and optimization
#         loss = torch.sum(loss)
#         loss.backward()
#         optimizer.step()

#         # Accumulate loss for the epoch
#         total_loss += loss.item()

#         # Update total sentences count
#         total_sentences += sentence_in.size(0)

#     # Calculate average loss for the epoch
#     average_loss = total_loss / total_sentences

#     # Print loss for each epoch
#     print(f'Epoch {epoch + 1}/{300}, Loss: {average_loss:.4f}')


In [26]:
torch.tensor([1,2,3])==torch.tensor([1,5,3])

tensor([ True, False,  True])

In [27]:
# !pip install wandb

In [28]:
import wandb
from tqdm import tqdm
import torch
from sklearn.metrics import f1_score

# Initialize WandB
wandb.init(project='NLP_AS2-Q3-P1', name='FastT-D2-ff-cc1', config={'epoch': 150, 'batch_size': 256})

# Assuming you have a BiLSTM_CRF model, a train_dataloader, a val_dataloader, and an optimizer
# Also assuming you have defined the necessary variables (e.g., vocab_size, tag_to_ix, etc.)

# Move the model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Function to calculate accuracy
import torch

def calculate_accuracy(predictions, targets, sen_lengths):
    ranges = targets.shape[0]
    target = targets.cpu()
    predictions = torch.tensor(predictions).cpu()
    acc = 0

    for i in range(ranges):
        prex = predictions[i][:sen_lengths[i]+1]
        trex = target[i][:sen_lengths[i]+1]
        acc += torch.sum(prex == trex)

    # Move the division outside the loop to calculate the average accuracy
    acc = acc.float() / (sum(sen_lengths)+10)
    # print(acc)
    return acc

def aggregater(predictions, targets, sen_lengths):
    ranges = targets.shape[0]
    target = targets.cpu()
    predictions = torch.tensor(predictions).cpu()
    acc = 0
    aggr_pred = []
    aggr_targ = []
    for i in range(ranges):
        prex = predictions[i][:sen_lengths[i]]
        trex = target[i][:sen_lengths[i]]
        aggr_pred.extend(prex)
        aggr_targ.extend(trex)
    return aggr_pred,aggr_targ

# WandB Config
config = wandb.config

# Watch the model
wandb.watch(model)

for epoch in range(config.epoch):
    # Training
    model.train()
    total_loss_train = 0
    correct_predictions_train = 0
    total_sentences_train = 0
    predictions_q = []
    traget_q = []

    for sentence_in, targets, mask, sen_lengths in tqdm(dataloader, desc=f'Training Epoch {epoch + 1}/{config.epoch}', leave=False):
        sentence_in, targets, mask, sen_lengths = sentence_in.to(device), targets.to(device), mask.to(device), sen_lengths.to(device)

        model.zero_grad()

        # Forward pass
        loss = model(sentence_in, mask, targets, sen_lengths)

        # Backward pass and optimization
        loss = torch.sum(loss)
        loss.backward()
        optimizer.step()

        # print('loss ',loss)
        # Accumulate loss for the epoch
        total_loss_train += (loss.item()/torch.sum(sen_lengths))

        # Prediction
        predictions_train = model.predict(sentence_in, mask, sen_lengths)
        correct_predictions_train += calculate_accuracy(predictions_train, targets, sen_lengths)

        temp_pred,temp_trag = aggregater(predictions_train, targets, sen_lengths)

        predictions_q.extend(temp_pred)
        traget_q.extend(temp_trag)

        # Update total sentences count
        # total_sentences_train += sentence_in.size(0)

    # Calculate average loss and accuracy for training
    average_loss_train = total_loss_train / len(dataloader)
    accuracy_train = correct_predictions_train / len(dataloader)  # Average over all sentences, not just batches
    f1_Score= f1_score(traget_q, predictions_q, average="macro")

    # Log metrics to WandB
    wandb.log({'Train Loss': average_loss_train, 'Train Accuracy': accuracy_train, 'Train F1': f1_Score}, step=epoch)
    print(f'Training Epoch {epoch + 1}/{150}, Loss: {average_loss_train:.4f}, Accuracy: {accuracy_train:.4f}, F1: {f1_Score :.4f}')
    # Validation
    model.eval()
    total_loss_val = 0
    correct_predictions_val = 0
    total_sentences_val = 0
    predictions_p = []
    traget_p = []

    with torch.no_grad():
        for sentence_in, targets, mask, sen_lengths in tqdm(dataloader_val, desc=f'Validation Epoch {epoch + 1}/{config.epoch}', leave=False):
            sentence_in, targets, mask, sen_lengths = sentence_in.to(device), targets.to(device), mask.to(device), sen_lengths.to(device)

            # Forward pass
            loss_val = model(sentence_in, mask, targets, sen_lengths)
            total_loss_val += (torch.sum(loss_val).item()/torch.sum(sen_lengths))

            # Prediction
            predictions_val = model.predict(sentence_in, mask, sen_lengths)
            correct_predictions_val += calculate_accuracy(predictions_val, targets, sen_lengths)

            temp_pred,temp_trag = aggregater(predictions_val, targets, sen_lengths)

            predictions_p.extend(temp_pred)
            traget_p.extend(temp_trag)

            # Update total sentences count
            # total_sentences_val += sentence_in.size(0)

    # Calculate average loss and accuracy for validation
    average_loss_val = total_loss_val / len(dataloader_val)
    accuracy_val = correct_predictions_val / len(dataloader_val)  # Average over all sentences, not just batches
    f1_Score = f1_score(traget_p, predictions_p, average="macro")

    # Log metrics to WandB
    wandb.log({'Validation Loss': average_loss_val, 'Validation Accuracy': accuracy_val, 'Validation F1': f1_Score}, step=epoch)
    print(f'Validation Epoch {epoch + 1}/{150}, Loss: {average_loss_val:.4f}, Accuracy: {accuracy_val:.4f}, F1: {f1_Score:.4f}')
    print()


Train Accuracy,▁▇▅█
Train F1,▁▆▆█
Train Loss,█▄▁▂
Validation Accuracy,▁███
Validation F1,▁▆██
Validation Loss,█▁▂▁
Train Accuracy,0.89727
Train F1,0.36887
Train Loss,5.9937
Validation Accuracy,0.95428
Validation F1,0.58919


Training Epoch 1/150, Loss: 12.9283, Accuracy: 0.3696, F1: 0.1201


Validation Epoch 1/150, Loss: 10.0331, Accuracy: 0.8546, F1: 0.1574



Training Epoch 2/150, Loss: 12.7435, Accuracy: 0.7281, F1: 0.2089


Validation Epoch 2/150, Loss: 6.0766, Accuracy: 0.8579, F1: 0.1535



...

Validation Epoch 148/150, Loss: 0.1091, Accuracy: 0.9961, F1: 0.8581



Training Epoch 149/150, Loss: 0.1352, Accuracy: 0.9867, F1: 0.6998


Validation Epoch 149/150, Loss: 0.1063, Accuracy: 0.9991, F1: 0.8728



Training Epoch 150/150, Loss: 0.1310, Accuracy: 0.9827, F1: 0.6880


Validation Epoch 150/150, Loss: 0.1085, Accuracy: 0.9974, F1: 0.8625



In [29]:
# from tqdm import tqdm
# import torch
# from sklearn.metrics import f1_score

# # Assuming you have a BiLSTM_CRF model, a train_dataloader, a val_dataloader, and an optimizer
# # Also assuming you have defined the necessary variables (e.g., vocab_size, tag_to_ix, etc.)

# # Move the model to GPU
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

# # Function to calculate accuracy
# import torch

# def calculate_accuracy(predictions, targets, sen_lengths):
#     ranges = targets.shape[0]
#     target = targets.cpu()
#     predictions = torch.tensor(predictions).cpu()
#     acc = 0

#     for i in range(ranges):
#         prex = predictions[i][:sen_lengths[i]]
#         trex = target[i][:sen_lengths[i]]
#         acc += torch.sum(prex == trex)

#     # Move the division outside the loop to calculate the average accuracy
#     acc = acc.float() / sum(sen_lengths)
#     # print(acc)
#     return acc

# def aggregater(predictions, targets, sen_lengths):
#     ranges = targets.shape[0]
#     target = targets.cpu()
#     predictions = torch.tensor(predictions).cpu()
#     acc = 0
#     aggr_pred = []
#     aggr_targ = []
#     for i in range(ranges):
#         prex = predictions[i][:sen_lengths[i]]
#         trex = target[i][:sen_lengths[i]]
#         aggr_pred.extend(prex)
#         aggr_targ.extend(trex)
#     return aggr_pred,aggr_targ


# for epoch in range(300):
#     # Training
#     model.train()
#     total_loss_train = 0
#     correct_predictions_train = 0
#     total_sentences_train = 0
#     predictions_q = []
#     traget_q = []

#     for sentence_in, targets, mask, sen_lengths in tqdm(dataloader, desc=f'Training Epoch {epoch + 1}/{300}', leave=False):
#         sentence_in, targets, mask, sen_lengths = sentence_in.to(device), targets.to(device), mask.to(device), sen_lengths.to(device)

#         model.zero_grad()

#         # Forward pass
#         loss = model(sentence_in, mask, targets, sen_lengths)

#         # Backward pass and optimization
#         loss = torch.sum(loss)
#         loss.backward()
#         optimizer.step()

#         # print('loss ',loss)
#         # Accumulate loss for the epoch
#         total_loss_train += (loss.item()/torch.sum(sen_lengths))

#         # Prediction
#         predictions_train = model.predict(sentence_in, mask, sen_lengths)
#         correct_predictions_train += calculate_accuracy(predictions_train, targets, sen_lengths)

#         temp_pred,temp_trag = aggregater(predictions_train, targets, sen_lengths)

#         predictions_q.extend(temp_pred)
#         traget_q.extend(temp_trag)

#         # Update total sentences count
#         # total_sentences_train += sentence_in.size(0)

#     # Calculate average loss and accuracy for training

#     average_loss_train = total_loss_train / len(dataloader)
#     accuracy_train = correct_predictions_train / len(dataloader)  # Average over all sentences, not just batches
#     f1_Score= f1_score(traget_q, predictions_q, average="macro")
#     print(f'Training Epoch {epoch + 1}/{300}, Loss: {average_loss_train:.4f}, Accuracy: {accuracy_train:.4f}, F1: {f1_Score :.4f}')

#     # Validation
#     model.eval()
#     total_loss_val = 0
#     correct_predictions_val = 0
#     total_sentences_val = 0
#     predictions_p = []
#     traget_p = []
#     with torch.no_grad():
#         for sentence_in, targets, mask, sen_lengths in tqdm(dataloader_val, desc=f'Validation Epoch {epoch + 1}/{300}', leave=False):
#             sentence_in, targets, mask, sen_lengths = sentence_in.to(device), targets.to(device), mask.to(device), sen_lengths.to(device)

#             # Forward pass
#             loss_val = model(sentence_in, mask, targets, sen_lengths)
#             total_loss_val += (torch.sum(loss_val).item()/torch.sum(sen_lengths))

#             # Prediction
#             predictions_val = model.predict(sentence_in, mask, sen_lengths)
#             correct_predictions_val += calculate_accuracy(predictions_val, targets, sen_lengths)


#             temp_pred,temp_trag = aggregater(predictions_val, targets, sen_lengths)

#             predictions_p.extend(temp_pred)
#             traget_p.extend(temp_trag)

#             # Update total sentences count
#             # total_sentences_val += sentence_in.size(0)

#     # Calculate average loss and accuracy for validation
#     average_loss_val = total_loss_val / len(dataloader_val)
#     accuracy_val = correct_predictions_val / len(dataloader_val)  # Average over all sentences, not just batches
#     f1_Score = f1_score(traget_p, predictions_p, average="macro")
#     print(f'Validation Epoch {epoch + 1}/{300}, Loss: {average_loss_val:.4f}, Accuracy: {accuracy_val:.4f}, F1: {f1_Score:.4f}')

#     print()

# # # Training and validation loop
# # for epoch in range(300):
# #     # Training
# #     model.train()
# #     total_loss_train = 0
# #     correct_predictions_train = 0
# #     total_sentences_train = 0

# #     for sentence_in, targets, mask, sen_lengths in tqdm(dataloader, desc=f'Training Epoch {epoch + 1}/{300}', leave=False):
# #         sentence_in, targets, mask, sen_lengths = sentence_in.to(device), targets.to(device), mask.to(device), sen_lengths.to(device)

# #         model.zero_grad()

# #         # Forward pass
# #         loss = model(sentence_in, mask, targets, sen_lengths)

# #         # Backward pass and optimization
# #         loss = torch.sum(loss)
# #         loss.backward()
# #         optimizer.step()

# #         # Accumulate loss for the epoch
# #         total_loss_train += loss.item()

# #         # Prediction
# #         predictions_train = model.predict(sentence_in, mask, sen_lengths)
# #         print()
# #         # print(len(predictions_train[0]),'predi_len')
# #         # print(sen_lengths[0],'sen_len')
# #         # print(targets[0][:sen_lengths[0]].shape,'targets_len')
# #         correct_predictions_train += calculate_accuracy(predictions_train, targets, sen_lengths)

# #         # Update total sentences count
# #         total_sentences_train += sentence_in.size(0)

# #     # Calculate average loss and accuracy for training
# #     average_loss_train = total_loss_train / total_sentences_train
# #     accuracy_train = correct_predictions_train / len(dataloader)

# #     print(f'Training Epoch {epoch + 1}/{300}, Loss: {average_loss_train:.4f}, Accuracy: {accuracy_train:.4f}')

# #     # Validation
# #     model.eval()
# #     total_loss_val = 0
# #     correct_predictions_val = 0
# #     total_sentences_val = 0

# #     with torch.no_grad():
# #         for sentence_in, targets, mask, sen_lengths in tqdm(dataloader_val, desc=f'Validation Epoch {epoch + 1}/{300}', leave=False):
# #             sentence_in, targets, mask, sen_lengths = sentence_in.to(device), targets.to(device), mask.to(device), sen_lengths.to(device)

# #             # Forward pass
# #             loss_val = model(sentence_in, mask, targets, sen_lengths)
# #             total_loss_val += torch.sum(loss_val).item()

# #             # Prediction
# #             predictions_val = model.predict(sentence_in, mask, sen_lengths)
# #             correct_predictions_val += calculate_accuracy(predictions_val, targets, sen_lengths)

# #             # Update total sentences count
# #             total_sentences_val += sentence_in.size(0)

# #     # Calculate average loss and accuracy for validation
# #     average_loss_val = total_loss_val / total_sentences_val
# #     accuracy_val = correct_predictions_val / len(dataloader_val)

# #     print(f'Validation Epoch {epoch + 1}/{300}, Loss: {average_loss_val:.4f}, Accuracy: {accuracy_val:.4f}')


In [30]:
# prompt: pytorch save model code

torch.save(model.state_dict(), 'model_d2_ff.pth')

# Save the model to W&B
wandb.save("'model_d2_ff.pth'")

[]

# # TESTING PHASE

In [1]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence


class BiLSTMCRF(nn.Module):
    def __init__(self, tag_vocab, dropout_rate=0.5, embed_size=300, hidden_size=256):
        """ Initialize the model
        Args:
            sent_vocab (Vocab): vocabulary of words
            tag_vocab (Vocab): vocabulary of tags
            embed_size (int): embedding size
            hidden_size (int): hidden state size
        """
        super(BiLSTMCRF, self).__init__()

        self.dropout_rate = dropout_rate
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        # self.sent_vocab = sent_vocab
        self.tag_vocab = tag_vocab
        # self.embedding = nn.Embedding(len(sent_vocab), embed_size) print
        self.dropout = nn.Dropout(dropout_rate)
        self.encoder = nn.LSTM(input_size=embed_size, hidden_size=hidden_size, bidirectional=True)
        self.hidden2emit_score = nn.Linear(hidden_size * 2, len(self.tag_vocab))
        self.transition = nn.Parameter(torch.randn(len(self.tag_vocab), len(self.tag_vocab)))  # shape: (K, K)

    def forward(self, sentences,mask, tags, sen_lengths):
        """
        Args:
            sentences (tensor): sentences, shape (b, len). Lengths are in decreasing order, len is the length
                                of the longest sentence
            tags (tensor): corresponding tags, shape (b, len)
            sen_lengths (list): sentence lengths
        Returns:
            loss (tensor): loss on the batch, shape (b,)
        """
        # mask = (sentences != self.sent_vocab[self.sent_vocab.PAD])  # shape: (b, len)                        #$$$$$$$$$$$$$$$$$$$__________________
        sentences = sentences.transpose(0, 1)  # shape: (len, b)
        # print("forword--1",sentences.shape)
        # sentences = self.embedding(sentences)  # shape: (len, b, e)
        emit_score = self.encode(sentences, sen_lengths)  # shape: (b, len, K)
        # print("forword--2",sentences.shape)
        loss = self.cal_loss(tags, mask, emit_score)  # shape: (b,)
        return loss

    def encode(self, sentences, sent_lengths):
        """ BiLSTM Encoder
        Args:
            sentences (tensor): sentences with word embeddings, shape (len, b, e)
            sent_lengths (list): sentence lengths
        Returns:
            emit_score (tensor): emit score, shape (b, len, K)
        """
        # padded_sentences = pack_padded_sequence(sentences, sent_lengths)
        hidden_states, _ = self.encoder(sentences)
        # print(hidden_states.shape,"(((())))")
        hidden_states = hidden_states.permute(1,0,2)
        # hidden_states, _ = pad_packed_sequence(hidden_states, batch_first=True)  # shape: (b, len, 2h)
        # print(hidden_states.shape)
        emit_score = self.hidden2emit_score(hidden_states)  # shape: (b, len, K)
        emit_score = self.dropout(emit_score)  # shape: (b, len, K)
        return emit_score

    # def encode(self, sentences, sent_lengths):
    #   """ BiLSTM Encoder
    #   Args:
    #       sentences (tensor): sentences with word embeddings, shape (len, b, e)
    #       sent_lengths (list): sentence lengths
    #   Returns:
    #       emit_score (tensor): emit score, shape (b, len, K)
    #   """
    #   sorted_lengths, sorted_idx = torch.sort(sent_lengths, descending=True)
    #   sorted_sentences = sentences[:, sorted_idx, :]  # Sort the sentences based on lengths
    #   packed_sentences = pack_padded_sequence(sorted_sentences, sorted_lengths)
    #   hidden_states, _ = self.encoder(packed_sentences)
    #   hidden_states, _ = pad_packed_sequence(hidden_states, batch_first=True)  # shape: (b, len, 2h)
    #   emit_score = self.hidden2emit_score(hidden_states)  # shape: (b, len, K)
    #   emit_score = self.dropout(emit_score)  # shape: (b, len, K)
    #   return emit_score

    def cal_loss(self, tags, mask, emit_score):
        """ Calculate CRF loss
        Args:
            tags (tensor): a batch of tags, shape (b, len)
            mask (tensor): mask for the tags, shape (b, len), values in PAD position is 0
            emit_score (tensor): emit matrix, shape (b, len, K)
        Returns:
            loss (tensor): loss of the batch, shape (b,)
        """
        batch_size, sent_len = tags.shape
        # calculate score for the tags
        score = torch.gather(emit_score, dim=2, index=tags.unsqueeze(dim=2)).squeeze(dim=2)  # shape: (b, len)
        score[:, 1:] += self.transition[tags[:, :-1], tags[:, 1:]]
        total_score = (score * mask.type(torch.float)).sum(dim=1)  # shape: (b,)
        # calculate the scaling factor
        d = torch.unsqueeze(emit_score[:, 0], dim=1)  # shape: (b, 1, K)
        fix_length = 100
        # for i in range(1, sent_len):
        for i in range(1, fix_length):
            n_unfinished = mask[:, i].sum()
            d_uf = d[: n_unfinished]  # shape: (uf, 1, K)
            emit_and_transition = emit_score[: n_unfinished, i].unsqueeze(dim=1) + self.transition  # shape: (uf, K, K)
            log_sum = d_uf.transpose(1, 2) + emit_and_transition  # shape: (uf, K, K)
            max_v = log_sum.max(dim=1)[0].unsqueeze(dim=1)  # shape: (uf, 1, K)
            log_sum = log_sum - max_v  # shape: (uf, K, K)
            d_uf = max_v + torch.logsumexp(log_sum, dim=1).unsqueeze(dim=1)  # shape: (uf, 1, K)
            d = torch.cat((d_uf, d[n_unfinished:]), dim=0)
        d = d.squeeze(dim=1)  # shape: (b, K)
        max_d = d.max(dim=-1)[0]  # shape: (b,)
        d = max_d + torch.logsumexp(d - max_d.unsqueeze(dim=1), dim=1)  # shape: (b,)
        llk = total_score - d  # shape: (b,)
        loss = -llk  # shape: (b,)
        return loss


    def predict(self, sentences, mask, sen_lengths):
        """
        Args:
            sentences (tensor): sentences, shape (b, len). Lengths are in decreasing order, len is the length
                                of the longest sentence
            sen_lengths (list): sentence lengths
        Returns:
            tags (list[list[str]]): predicted tags for the batch
        """
        batch_size = sentences.shape[0]

        w = mask
        sentences = sentences.transpose(0, 1)

        emit_score = self.encode(sentences, sen_lengths)

        # Initialize the tags with all possible tag indices for each sentence in the batch
        tags = [[[i] for i in range(len(self.tag_vocab))]] * batch_size  # list, shape: (b, K, 1)

        # Initialize the first column of the dynamic programming matrix
        d = torch.unsqueeze(emit_score[:, 0], dim=1)  # shape: (b, 1, K)

        # Use a fixed length (e.g., 100) instead of max(sen_lengths)
        fixed_length = 100

        # Iterate over the remaining columns of the dynamic programming matrix
        for i in range(1, fixed_length):
            # Calculate the number of unfinished sentences at the current position
            n_unfinished = mask[:, i].sum()

            # Slice the dynamic programming matrix for the unfinished sentences
            d_uf = d[: n_unfinished]  # shape: (uf, 1, K)

            # Compute emission and transition scores for the current position
            emit_and_transition = self.transition + emit_score[: n_unfinished, i].unsqueeze(dim=1)  # shape: (uf, K, K)

            # Compute the new values for the dynamic programming matrix
            new_d_uf = d_uf.transpose(1, 2) + emit_and_transition  # shape: (uf, K, K)

            # Update the dynamic programming matrix and get the indices of maximum values
            d_uf, max_idx = torch.max(new_d_uf, dim=1)
            max_idx = max_idx.tolist()  # list, shape: (nf, K)

            # Update the tags for the unfinished sentences
            tags[: n_unfinished] = [[tags[b][k] + [j] for j, k in enumerate(max_idx[b])] for b in range(n_unfinished)]

            # Concatenate the new values to the dynamic programming matrix
            d = torch.cat((torch.unsqueeze(d_uf, dim=1), d[n_unfinished:]), dim=0)  # shape: (b, 1, K)

        # Remove the singleton dimension to get the final dynamic programming matrix
        d = d.squeeze(dim=1)  # shape: (b, K)

        # Get the indices of the maximum values in the final column of the matrix
        _, max_idx = torch.max(d, dim=1)  # shape: (b,)
        max_idx = max_idx.tolist()

        # Extract the predicted tags based on the maximum indices
        tags = [tags[b][k] for b, k in enumerate(max_idx)]

        # Print the predicted tags and sentence lengths for debugging
        # print(tags, sen_lengths, '((()))')

        return tags

tag_to_ix ={'O': 0, 'B': 1, 'I':2,'<START>':3,'<STOP>':4,'<PAD>':5}
# tag_vocab, dropout_rate=0.5, embed_size=300, hidden_size=256)
model  = BiLSTMCRF(tag_to_ix,dropout_rate=0.5, embed_size=300, hidden_size=256)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Function to calculate accuracy
import torch

def calculate_accuracy(predictions, targets, sen_lengths):
    ranges = targets.shape[0]
    target = targets.cpu()
    predictions = torch.tensor(predictions).cpu()
    acc = 0

    for i in range(ranges):
        prex = predictions[i][:sen_lengths[i]+1]
        trex = target[i][:sen_lengths[i]+1]
        acc += torch.sum(prex == trex)

    # Move the division outside the loop to calculate the average accuracy
    acc = acc.float() / (sum(sen_lengths)+10)
    # print(acc)
    return acc

def aggregater(predictions, targets, sen_lengths):
    ranges = targets.shape[0]
    target = targets.cpu()
    predictions = torch.tensor(predictions).cpu()
    acc = 0
    aggr_pred = []
    aggr_targ = []
    for i in range(ranges):
        prex = predictions[i][:sen_lengths[i]]
        trex = target[i][:sen_lengths[i]]
        aggr_pred.extend(prex)
        aggr_targ.extend(trex)
    return aggr_pred,aggr_targ
import torch
from torch.utils.data import Dataset,DataLoader
import gensim.downloader as api
from torchtext.vocab import GloVe
import fasttext
import numpy as np
from torchtext.vocab import GloVe,FastText
import fasttext.util
import json
class SentimentAnalysisDataset(Dataset):
    def __init__(self, json_path, embedding_type='word2vec',load=True):
        with open(json_path, 'r') as file:
            self.data = json.load(file)

        self.embedding_type = embedding_type
        if load:
          self.embedding_model =self.load_embedding_model()
        else:
          self.embedding_model = None

    def load_embedding_model(self):
        if self.embedding_type == 'word2vec':
            # Download the pre-trained Word2Vec model
            return api.load('word2vec-google-news-300')
        elif self.embedding_type == 'glove':
            # Download the pre-trained GloVe model (6B tokens, 300d)
            return GloVe(name='6B', dim=300)
        elif self.embedding_type == 'fasttext':
            # Load the pre-trained FastText model
            return FastText(language='en')
        else:
            raise ValueError(f"Unsupported embedding type: {self.embedding_type}")

    def text_to_embeddings(self, text):
        maxlen = 100
        if self.embedding_type == 'word2vec':
            # Word2Vec embeddings
            embeddings = [self.embedding_model[word] if word in self.embedding_model else torch.zeros(self.embedding_model.vector_size) for word in text.split() ]
            # print(np.stack(embeddings).shape)
            embeddings = [torch.full((self.embedding_model.vector_size,),-1000.0)] + embeddings + [torch.full((self.embedding_model.vector_size,),+1000.0)]

            # print('##',np.stack([torch.full((self.embedding_model.vector_size,),-1000.0)] + embeddings + [torch.full((self.embedding_model.vector_size,),+1000.0)]  ).shape)

            for i in range(100-len(embeddings)):
              embeddings.append(torch.full((self.embedding_model.vector_size,),-1.0))


        elif self.embedding_type == 'glove':
            # GloVe embeddings
            embeddings = [self.embedding_model[word] if word in self.embedding_model else torch.zeros(sentiment_dataset.embedding_model['a'].shape[0]) for word in text.split() ]
            # print(np.stack(embeddings).shape)
            embeddings = [torch.full((sentiment_dataset.embedding_model['a'].shape[0],),-1000.0)] + embeddings + [torch.full((sentiment_dataset.embedding_model['a'].shape[0],),+1000.0)]

            # print('##',np.stack([torch.full((self.embedding_model.vector_size,),-1000.0)] + embeddings + [torch.full((self.embedding_model.vector_size,),+1000.0)]  ).shape)

            for i in range(100-len(embeddings)):
              embeddings.append(torch.full((sentiment_dataset.embedding_model['a'].shape[0],),-1.0))

        elif self.embedding_type == 'fasttext':
            # FastText embeddings
            embeddings = [self.embedding_model[word] for word in text.split() ]
            # print(np.stack(embeddings).shape)
            embeddings = [torch.full((sentiment_dataset.embedding_model['a'].shape[0],),-1000.0)] + embeddings + [torch.full((sentiment_dataset.embedding_model['a'].shape[0],),+1000.0)]

            # print('##',np.stack([torch.full((self.embedding_model.vector_size,),-1000.0)] + embeddings + [torch.full((self.embedding_model.vector_size,),+1000.0)]  ).shape)

            for i in range(100-len(embeddings)):
              embeddings.append(torch.full((sentiment_dataset.embedding_model['a'].shape[0],),-1.0))
        else:
            raise ValueError(f"Unsupported embedding type: {self.embedding_type}")
        # print()
        return np.stack(embeddings)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):


        index = str(index)
        text = self.data[index]["text"]
        labels = self.data[index]["labels"]

        # Convert text to embeddings
        text_embeddings = torch.tensor(self.text_to_embeddings(text))
        # print(text_embeddings.shape)
        # torch.stack([torch.full((1,text_embeddings.shape[1]),-1000),text_embeddings, [torch.full((1,text_embeddings.shape[1]),1000)])
        current_length = len(labels)

        labels = ['<START>'] + labels + ['<STOP>']

        sent_lengths =torch.tensor(len(labels))

        max_length = 100
        labels = labels + ['<PAD>'] * (max_length - (current_length+2))

        # Convert labels to numerical format if needed
        label_mapping = {'O': 0, 'B': 1, 'I':2,'<START>':3,'<STOP>':4,'<PAD>':5}                               # bind it to self__________________________
        numerical_labels = [label_mapping[label] for label in labels ]


        # Pad the sequence to the maximum length

        # Convert labels to PyTorch tensor
        labels_tensor = torch.tensor(numerical_labels)
        mask = torch.hstack([torch.full((text_embeddings.shape[0],),True),torch.full((100-text_embeddings.shape[0],),False)])
        # print(labels_tensor.shape,text_embeddings.shape,mask.shape)
        return text_embeddings, labels_tensor, mask,sent_lengths




In [4]:
model_state_dict = torch.load('t2_model4_fasttext.pt')

# Load the state dictionary into the model
model.load_state_dict(model_state_dict)

json_path = 'ATE_test.json'
embedding_type = 'fasttext'
sentiment_dataset_test = SentimentAnalysisDataset(json_path, embedding_type)
sentiment_dataset =sentiment_dataset_test
# sentiment_dataset_test.embedding_model = sentiment_dataset.embedding_model
batch_size  = 512
dataloader_test = DataLoader(sentiment_dataset_test, batch_size=batch_size, shuffle=True)

In [5]:
from tqdm import tqdm
from sklearn.metrics import f1_score
model.eval()
correct_predictions_val = 0
total_sentences_val = 0
predictions_r = []
traget_r = []
epoch=1
device='cuda'
with torch.no_grad():
    for sentence_in, targets, mask, sen_lengths in tqdm(dataloader_test, desc=f'Test Epoch {epoch + 1}/{300}', leave=False):
        sentence_in, targets, mask, sen_lengths = sentence_in.to(device), targets.to(device), mask.to(device), sen_lengths.to(device)

        # Prediction
        predictions_val = model.predict(sentence_in, mask, sen_lengths)
        correct_predictions_val += calculate_accuracy(predictions_val, targets, sen_lengths)
        temp_pred,temp_trag = aggregater(predictions_val, targets, sen_lengths)
        predictions_r.extend(temp_pred)
        traget_r.extend(temp_trag)

accuracy_val = correct_predictions_val / len(dataloader_test)  # Average over all sentences, not just batches
print()
print(f'Test Accuracy: {accuracy_val:.4f}')
print(f'Test F1:  {f1_score(traget_r, predictions_r, average="macro")}')



Test Accuracy: 0.9786
Test F1:  0.8216757024437102


In [33]:
wandb.finish()

Train Accuracy,▁▃▃▅▆▇▇▇▇▇▇▇▇▇▇▇▇███████████████████████
Train F1,▁▄▄▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇██▇██████████████
Train Loss,█▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation Accuracy,▁▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇█▇█▇█▇▇██▇████▇████████
Validation F1,▁▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇█▇█▇█▇▇██▇████▇████████
Validation Loss,█▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train Accuracy,0.9827
Train F1,0.68799
Train Loss,0.13097
Validation Accuracy,0.9974
Validation F1,0.8625
